# 1. Ask an interesting question

1. Delay binary classifier (yes/no, >15 min)
- Better isolate the key features to predict delays so that airlines focus their efforts on their weakest points
- Account for uncontrollable variables (e.g. unpredicted adverse weather) and prepare to act consequently
2. Additional questions:
- When is the best time of day/day of week/time of year to fly to minimize delays?
- Do older planes suffer more delays?
- How does the number of people flying between different locations change over time?
- How well does weather predict plane delays?
- Can you detect cascading failures as delays in one airport create delays in others? Are there critical links in the system?

___

# 2. Get the data

### On-Time : Reporting Carrier On-Time Performance (1987-present)

Source: https://www.transtats.bts.gov/DatabaseInfo.asp?DB_ID=120&DB_URL=Mode_ID=1&Mode_Desc=Aviation&Subject_ID2=0

<em>Note</em>: Over time both the code and the name of a carrier may change and the same code or name may be assumed by a different airline. To ensure that you are analyzing data from the same airline, TranStats provides four airline-specific variables that identify one and only one carrier or its entity: Airline ID (AirlineID), Unique Carrier Code (UniqueCarrier), Unique Carrier Name (UniqueCarrierName), and Unique Entity (UniqCarrierEntity). A unique airline (carrier) is defined as one holding and reporting under the same DOT certificate regardless of its Code, Name, or holding company/corporation. US Airways and America West started to report combined on-time data in January 2006 and combined traffic and financial data in October 2007 following their 2005 merger announcement. Delta and Northwest began reporting jointly in January 2010 following their 2008 merger announcement. Continental Micronesia was combined into Continental Airlines in December 2010 and joint reporting began in January 2011. Atlantic Southeast and ExpressJet began reporting jointly in January 2012. United and Continental began reporting jointly in January 2012 following their 2010 merger announcement. Endeavor (9E) operated as Pinnacle prior to August 2013. Envoy (MQ) operated as American Eagle prior to April 2014. Southwest (WN) and AirTran (FL) began reporting jointly in January 2015 following their 2011 merger announcement. American (AA) and US Airways (US) began reporting jointly as AA in July 2015 following their 2013 merger announcement. Alaska (AS) and Virgin America (VX) began reporting jointly as AS in April 2018 following their 2016 merger announcement.

        
- **AC_Types**:
    - **Summary**:
        - Reporting Carrier On-Time Performance (1987-present)
    - **Description**:
        - Reporting carriers are required to (or voluntarily) report on-time data for flights they operate: on-time arrival and departure data for non-stop domestic flights by month and year, by carrier and by origin and destination airport. Includes scheduled and actual departure and arrival times, canceled and diverted flights, taxi-out and taxi-in times, causes of delay and cancellation, air time, and non-stop distance. Use Download for individual flight data.
    - **File**:
        - YYMM_123456789_T_ONTIME_REPORTING.zip *(YY = Year ; MM = Month)*

In [ ]:
# Import libraries to be used

# Warning messages display
## import warnings
## warnings.filterwarnings(action='ignore') # https://docs.python.org/3/library/warnings.html#the-warnings-filter

# Directories/Files management
import os.path
## from zipfile import ZipFile # De momento no ha hecho falta 

# Timing
import time

# Memory monitoring
%load_ext memory_profiler
### Use '%memit' to check at each point

# Data analysis and wrangling
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None) # Show all columns in DataFrames
## pd.set_option('display.max_rows', None) # It greatly slows down the output display and freezes the kernel

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('ggplot') # choose a style: 'plt.style.available'
sns.set_theme(context='notebook',
              style="darkgrid") # {darkgrid, whitegrid, dark, white, ticks}
palette = sns.color_palette("flare", as_cmap=True);
import altair as alt

# Machine Learning
## from sklearn.[...] import ...

In [ ]:
t0 = time.perf_counter() 

In [ ]:
# Detect Operating System running and manage paths accordingly

if os.name == 'nt': # Windows
    root = r"C:\Users\turge\CompartidoVM\0.TFM"
    print("Running on Windows.")
elif os.name == 'posix': # Ubuntu
    root = "/home/dsc/shared/0.TFM"
    print("Running on Ubuntu.")
print("root path\t", root)

___

### SAMPLE FILE

Single file import (i.e. month-sized database)

In [ ]:
csv_path = os.path.join(root,
                        "Raw_Data",
                        "US_DoT",
                        "ONTIME_REPORTING",
                        "1901_921771952_T_ONTIME_REPORTING.zip")
csv_path

Considering that each file is considerably big, and that later on many of them will have to be grouped, a first exploration will be done considering the first 10,000 rows.

In [ ]:
# Since 'pd.read_csv' works fine with zipped csv files, we can proceed directly:
cols = pd.read_csv(csv_path, nrows=1).columns # After normally importing it, an undesired extra blank column is loaded
df_0 = pd.read_csv(csv_path,
                  encoding='latin1',
                  nrows=10000,
                  usecols=cols[:-1], # This way, the extra column is disregarded for the loading process
                  low_memory = False)
df_0.sample(5)

Let's see which columns will remain, and skip the rest.

___

### MULTIPLE FILE

Let's proceed with multiple-file importing, through concatenation into a single DataFrame.

___

#### Run only the first time to read the 12 month individual files and generate the global file (entire year 2019)

##### 1. Read individual month files

##### 2. Generate file (2019)

___

In [ ]:
%memit

In [ ]:
csv_path = os.path.join(root,
                        "Output_Data",
                        "US_DoT",
                        "AL_OTP_MVP_Preprocessed_19_v1.csv")
csv_path

In [ ]:
%%time

# Chunk-loading procedure is required so as to prevent memory-saturation errors from appearing:

chunks_list = []
chunks = pd.read_csv(csv_path,
                     encoding='latin1',
#                      nrows=1e6, # Fail-safe: in case the file is inadvertently too big
                     chunksize=1e6,
                     usecols=cols[:-1], # This way, the extra column is disregarded for the loading process
                     low_memory = False)

for i, chunk in enumerate(chunks):
    if i == 13: # Fail-safe: for debugging purposes only
        break
    chunks_list.append(chunk)

    
df = pd.concat(chunks_list, axis=0)
del chunks_list

df.sample(5)

In [ ]:
df.shape

In [ ]:
%memit

There are some missing values. Let's further delve into it:

In [ ]:
%%time

# Absolute & Relative frequency of missing values by column:
pd.set_option('display.max_rows', 110) # It greatly slows down the output display and freezes the kernel
missing = pd.DataFrame([df.isna().sum(), df.isna().sum() / len(df) * 100], index=['Absolute', 'Relative']).T.sort_values(by='Relative', ascending=False)
missing

In [ ]:
# Show which columns present more than 5% of missing data:
empty_df = missing[missing['Relative'] > 5]
empty_df

In [ ]:
%%time

# Drop every column with more than 90% of missing values:
empty90_df = missing[missing['Relative'] > 90]
empty90_df_cols = empty90_df.index.to_list()
df = df.drop(empty90_df_cols, axis=1)
df.shape

In [ ]:
# CHECK AGAIN after manipulating the missing data.
# Show which columns present more than 5% of missing data:
empty_df = missing[missing['Relative'] > 5]
empty_df

In [ ]:
%%time

empty_df_cols = empty_df.index.to_list()
df[empty_df_cols] = df[empty_df_cols].fillna(value=0)

In [ ]:
%memit

In [ ]:
isolated_elements_missing = missing.loc[(missing['Relative'] < 5) & (missing['Relative'] > 0)]
isolated_elements_missing

In [ ]:
%%time

# Quick approach → check how many rows contain empty values to see if directly dropping them would be feasible:

# Check which rows have at least 1 NaN:
df[df.isna().any(axis=1)]

In [ ]:
%%time

# DF's length as is:
original_length = len(df)
print('Original dataset length:\t', original_length)

# Check how many rows have at least 1 NaN:
manipulated_length = len(df.drop(df[df.isna().any(axis=1)].index, axis=0))
print('Manipulated dataset length:\t', manipulated_length)

# Dropped rows, absolute and relative number:
print('{} rows deleted, accounting for {:.2f}% of the original dataset.'.format(original_length - manipulated_length, (original_length - manipulated_length) / original_length * 100))

Considering that it is only ~2% of the data, and that the dataset is big enough, let's just simply drop those rows as a quick  cleaning method.

In [ ]:
%%time

df = df.drop(df[df.isna().any(axis=1)].index, axis=0)
df

In [ ]:
%%time

# CHECK AGAIN after manipulating the missing data.
# Absolute & Relative frequency of missing values by column:
pd.set_option('display.max_rows', 110) # It greatly slows down the output display and freezes the kernel
missing_2 = pd.DataFrame([df.isna().sum(), df.isna().sum() / len(df) * 100], index=['Absolute', 'Relative']).T.sort_values(by='Relative', ascending=False)
missing_2

Great. Now the dataset is totally free of missing values.

Let's store the recently cleaned dataset in a CSV.

In [ ]:
pd.set_option('display.max_rows', 10) # Go back to the default display value (10 rows)

#### Run only the first time to generate the global CLEAN file (year 2019)

___

In [ ]:
int_cols = [
# Time Period
 'YEAR',
#  'QUARTER', # Disregarded: redundant
 'MONTH',
 'DAY_OF_MONTH',
 'DAY_OF_WEEK',
#  'FL_DATE', # Disregarded: redundant
# Airline / Aircraft
 'OP_UNIQUE_CARRIER',
 'OP_CARRIER_AIRLINE_ID',
 'OP_CARRIER',
 'TAIL_NUM',
 'OP_CARRIER_FL_NUM',
# Origin
 'ORIGIN_AIRPORT_ID',
 'ORIGIN_AIRPORT_SEQ_ID',
 'ORIGIN_CITY_MARKET_ID',
 'ORIGIN',
 'ORIGIN_CITY_NAME',
 'ORIGIN_STATE_ABR',
 'ORIGIN_STATE_FIPS',
 'ORIGIN_STATE_NM',
 'ORIGIN_WAC',
# Destination
 'DEST_AIRPORT_ID',
 'DEST_AIRPORT_SEQ_ID',
 'DEST_CITY_MARKET_ID',
 'DEST',
 'DEST_CITY_NAME',
 'DEST_STATE_ABR',
 'DEST_STATE_FIPS',
 'DEST_STATE_NM',
 'DEST_WAC',
# Departure Performance
 'CRS_DEP_TIME',
 'DEP_TIME',
 'DEP_DELAY',
 'DEP_DELAY_NEW',
 'DEP_DEL15',
 'DEP_DELAY_GROUP',
 'DEP_TIME_BLK',
 'TAXI_OUT',
 'WHEELS_OFF',
# Arrival Performance
 'WHEELS_ON',
 'TAXI_IN',
 'CRS_ARR_TIME',
 'ARR_TIME',
 'ARR_DELAY',
 'ARR_DELAY_NEW',
 'ARR_DEL15',
 'ARR_DELAY_GROUP',
 'ARR_TIME_BLK',
# Cancellations and Diversions
 'CANCELLED',
 'CANCELLATION_CODE',
 'DIVERTED',
# Flight Summaries
 'CRS_ELAPSED_TIME',
 'ACTUAL_ELAPSED_TIME',
 'AIR_TIME',
 'FLIGHTS',
 'DISTANCE',
 'DISTANCE_GROUP',
# Cause of Delay
 'CARRIER_DELAY',
 'WEATHER_DELAY',
 'NAS_DELAY',
 'SECURITY_DELAY',
 'LATE_AIRCRAFT_DELAY',
# Gate Return Information at Origin Airport (Data starts 10/2008)
 'FIRST_DEP_TIME',
 'TOTAL_ADD_GTIME',
 'LONGEST_ADD_GTIME',
# Diverted Airport Information (Data starts 10/2008)
 'DIV_AIRPORT_LANDINGS',
 'DIV_REACHED_DEST',
 'DIV_ACTUAL_ELAPSED_TIME',
 'DIV_ARR_DELAY',
 'DIV_DISTANCE',
 'DIV1_AIRPORT',
 'DIV1_AIRPORT_ID',
 'DIV1_AIRPORT_SEQ_ID',
 'DIV1_WHEELS_ON',
 'DIV1_TOTAL_GTIME',
 'DIV1_LONGEST_GTIME',
 'DIV1_WHEELS_OFF',
 'DIV1_TAIL_NUM',
 'DIV2_AIRPORT',
 'DIV2_AIRPORT_ID',
 'DIV2_AIRPORT_SEQ_ID',
 'DIV2_WHEELS_ON',
 'DIV2_TOTAL_GTIME',
 'DIV2_LONGEST_GTIME',
 'DIV2_WHEELS_OFF',
 'DIV2_TAIL_NUM',
 'DIV3_AIRPORT',
 'DIV3_AIRPORT_ID',
 'DIV3_AIRPORT_SEQ_ID',
 'DIV3_WHEELS_ON',
 'DIV3_TOTAL_GTIME',
 'DIV3_LONGEST_GTIME',
 'DIV3_WHEELS_OFF',
 'DIV3_TAIL_NUM',
 'DIV4_AIRPORT',
 'DIV4_AIRPORT_ID',
 'DIV4_AIRPORT_SEQ_ID',
 'DIV4_WHEELS_ON',
 'DIV4_TOTAL_GTIME',
 'DIV4_LONGEST_GTIME',
 'DIV4_WHEELS_OFF',
 'DIV4_TAIL_NUM',
 'DIV5_AIRPORT',
 'DIV5_AIRPORT_ID',
 'DIV5_AIRPORT_SEQ_ID',
 'DIV5_WHEELS_ON',
 'DIV5_TOTAL_GTIME',
 'DIV5_LONGEST_GTIME',
 'DIV5_WHEELS_OFF',
 'DIV5_TAIL_NUM'
]

In [ ]:
print("{} columns kept from a total of {}.".format(len(int_cols), len(cols)))

Additional information on each column meaning can be found [here](https://www.transtats.bts.gov/Fields.asp?Table_ID=236&SYS_Table_Name=T_ONTIME_REPORTING&User_Table_Name=Reporting%20Carrier%20On-Time%20Performance%20(1987-present)&Year_Info=1&First_Year=1987&Last_Year=2020&Rate_Info=0&Frequency=Monthly&Data_Frequency=Annual,Quarterly,Monthly).

___

# 3. Explore the data

## Plots

In [ ]:
cols = [
 'YEAR',
 'QUARTER',
 'MONTH',
 'DAY_OF_MONTH',
 'DAY_OF_WEEK',
#  'FL_DATE',
#  'OP_UNIQUE_CARRIER',
#  'OP_CARRIER_AIRLINE_ID',
#  'OP_CARRIER',
#  'TAIL_NUM',
#  'OP_CARRIER_FL_NUM',
#  'ORIGIN_AIRPORT_ID',
#  'ORIGIN_AIRPORT_SEQ_ID',
#  'ORIGIN_CITY_MARKET_ID',
#  'ORIGIN',
#  'ORIGIN_CITY_NAME',
#  'ORIGIN_STATE_ABR',
#  'ORIGIN_STATE_FIPS',
#  'ORIGIN_STATE_NM',
#  'ORIGIN_WAC',
#  'DEST_AIRPORT_ID',
#  'DEST_AIRPORT_SEQ_ID',
#  'DEST_CITY_MARKET_ID',
#  'DEST',
#  'DEST_CITY_NAME',
#  'DEST_STATE_ABR',
#  'DEST_STATE_FIPS',
#  'DEST_STATE_NM',
#  'DEST_WAC',
#  'CRS_DEP_TIME',
#  'DEP_TIME',
 'DEP_DELAY',
 'DEP_DELAY_NEW',
 'DEP_DEL15',
#  'DEP_DELAY_GROUP',
#  'DEP_TIME_BLK',
 'TAXI_OUT',
#  'WHEELS_OFF',
#  'WHEELS_ON',
 'TAXI_IN',
#  'CRS_ARR_TIME',
#  'ARR_TIME',
 'ARR_DELAY',
 'ARR_DELAY_NEW',
 'ARR_DEL15',
#  'ARR_DELAY_GROUP',
#  'ARR_TIME_BLK',
 'CANCELLED',
 'CANCELLATION_CODE',
 'DIVERTED',
#  'CRS_ELAPSED_TIME',
#  'ACTUAL_ELAPSED_TIME',
 'AIR_TIME',
 'FLIGHTS',
 'DISTANCE',
#  'DISTANCE_GROUP',
 'CARRIER_DELAY',
 'WEATHER_DELAY',
 'NAS_DELAY',
 'SECURITY_DELAY',
 'LATE_AIRCRAFT_DELAY',
#  'FIRST_DEP_TIME',
#  'TOTAL_ADD_GTIME',
#  'LONGEST_ADD_GTIME',
#  'DIV_AIRPORT_LANDINGS',
#  'DIV_REACHED_DEST',
#  'DIV_ACTUAL_ELAPSED_TIME',
 'DIV_ARR_DELAY',
 'DIV_DISTANCE',
#  'DIV1_AIRPORT',
#  'DIV1_AIRPORT_ID',
#  'DIV1_AIRPORT_SEQ_ID',
#  'DIV1_WHEELS_ON',
#  'DIV1_TOTAL_GTIME',
#  'DIV1_LONGEST_GTIME',
#  'DIV1_WHEELS_OFF',
#  'DIV1_TAIL_NUM',
#  'DIV2_AIRPORT',
#  'DIV2_AIRPORT_ID',
#  'DIV2_AIRPORT_SEQ_ID',
#  'DIV2_WHEELS_ON',
#  'DIV2_TOTAL_GTIME',
#  'DIV2_LONGEST_GTIME',
#  'DIV2_WHEELS_OFF',
#  'DIV2_TAIL_NUM',
#  'DIV3_AIRPORT',
#  'DIV3_AIRPORT_ID',
#  'DIV3_AIRPORT_SEQ_ID',
#  'DIV3_WHEELS_ON',
#  'DIV3_TOTAL_GTIME',
#  'DIV3_LONGEST_GTIME',
#  'DIV3_WHEELS_OFF',
#  'DIV3_TAIL_NUM',
#  'DIV4_AIRPORT',
#  'DIV4_AIRPORT_ID',
#  'DIV4_AIRPORT_SEQ_ID',
#  'DIV4_WHEELS_ON',
#  'DIV4_TOTAL_GTIME',
#  'DIV4_LONGEST_GTIME',
#  'DIV4_WHEELS_OFF',
#  'DIV4_TAIL_NUM',
#  'DIV5_AIRPORT',
#  'DIV5_AIRPORT_ID',
#  'DIV5_AIRPORT_SEQ_ID',
#  'DIV5_WHEELS_ON',
#  'DIV5_TOTAL_GTIME',
#  'DIV5_LONGEST_GTIME',
#  'DIV5_WHEELS_OFF',
#  'DIV5_TAIL_NUM'
]

In [ ]:
df0 = df0[cols]
df0

In [ ]:
output_csv_path = os.path.join(root,
                               "Output_Data",
                               "US_DoT",
                               "AL_OTP_MVP_Preprocessed.csv")
output_csv_path

In [ ]:
%%time

df0.to_csv(path_or_buf=output_csv_path,
           index=False,
           encoding='latin1')

In [ ]:
t1 = time.perf_counter()  - t0
print("Time elapsed: ", t1) # CPU seconds elapsed (floating point)

# 5. Communicate and visualize the results

___